Demo ipynb for LDA

Creating training pipeline on different situations (15/01/2024)

- For all games (using 0.1 and 0.5 of the whole dataset for pipeline developments)
- For top 11 genres

- (if possible): per game, focus on large and small (indie) games

Do the same thing for all three model architectures

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import json
import pickle
from datetime import datetime
import itertools

import gensim

import spacy

import nltk

import pyLDAvis

In [2]:
# download nltk stopwords
# import nltk
# nltk.download('stopwords')

# download spacy stopwords
# ...

In [3]:
# training conditions

from enum import Enum

class TRAINING_CONDS(Enum):
    ALL_GAMES = 1
    ALL_GAMES_LARGE = 2
    BY_GENRE = 3
    ALL_GAMES_TINY = 4
    BY_GAME = 5

# training condition
training_cond = TRAINING_CONDS.BY_GAME

In [4]:
# load a dataset

GENRES = ['Action', 'Indie', 'Adventure', 'RPG', 'Strategy', 'Simulation', 'Free to Play', 'Causal', 'Massively Multiplayer', 'Racing', 'Sports']
GAMES = ['Terraria', 'PAYDAY 2', 'Dota 2', 'Rocket League', 'Undertale', 'Left 4 Dead 2', 'Warframe', 'Portal 2', 'Grand Theft Auto V', 'Fallout: New Vegas']

training_genre_id = 9
training_game_id = 0

if training_cond == TRAINING_CONDS.ALL_GAMES:
    dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo.pkl')
elif training_cond == TRAINING_CONDS.ALL_GAMES_LARGE:
    dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo_large.pkl')
elif training_cond == TRAINING_CONDS.ALL_GAMES_TINY:
    dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo_tiny.pkl')
elif training_cond == TRAINING_CONDS.BY_GENRE:
    dataset_path = Path(f'../../dataset/topic_modelling/top_11_genres/{training_genre_id:02}_{GENRES[training_genre_id]}.pkl')
elif training_cond == TRAINING_CONDS.BY_GAME:
    dataset_path = Path(f'../../dataset/topic_modelling/top_10_games/{training_game_id:02}_{GAMES[training_game_id]}.pkl')

# dataset_path = Path('../../dataset/topic_modelling/top_10_games/00_Terraria.pkl')

# dataset_path = Path('../dataset_cleaned_heartless_sampled_for_demo.pkl')


dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 75499 entries, 57735 to 133233
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         75499 non-null  int64 
 1   app_id        75499 non-null  int64 
 2   app_name      75499 non-null  object
 3   review_text   75499 non-null  object
 4   review_score  75499 non-null  int64 
 5   review_votes  75499 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.0+ MB


In [5]:
%load_ext autoreload

In [6]:
# data preprocessing

import re

import sys
sys.path.append('../../sa')

%autoreload 2
import str_cleaning_functions

def cleaning(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_non_letters(x))
    df[review] = df[review].apply(lambda x: x.lower())
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_stopword(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

def cleaning_strlist(str_list):
    str_list = list(map(lambda x: str_cleaning_functions.remove_links(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.remove_links2(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.clean(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.deEmojify(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.remove_non_letters(x), str_list))
    str_list = list(map(lambda x: x.lower(), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.unify_whitespaces(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.remove_stopword(x), str_list))
    str_list = list(map(lambda x: str_cleaning_functions.unify_whitespaces(x), str_list))
    return str_list

In [7]:
# apply data preprocessing

cleaning(dataset, 'review_text')

In [8]:
X = dataset['review_text'].values

In [9]:
X

array(['werewolf riding unicorn shooting rainbows gun build teleporters find hair dresser spider cavern get sword shoots cats take lord moon using yoyo summon sharknado minion shoots sharks enemies find sky temples air wyverns spawn buy music box wizard go record music like playing base whenever want go build castle made entirely white marble would seem thing minecraft game dimension trust get used start learning game terraria simply one satisfying sandbox experiences may sound rude compared minecraft imagination',
       'copies game go around giving people look sad', 'introduction',
       ...,
       'game game start newb get pro hours entertainment even get bored get mods newb',
       'far one greatest games played yet',
       'game awesome eye cthulhu possible'], dtype=object)

In [10]:
# nltk.download('averaged_perceptron_tagger')
# t = nltk.word_tokenize(X[0])
# tt = nltk.pos_tag(t)
# tt

In [11]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [12]:
# lemmatize the data

X_lemmatized = list(map(lambda x: lemmatization(x), X))

In [13]:
X_lemmatized[0]

['werewolf',
 'rid',
 'unicorn',
 'shooting',
 'rainbow',
 'gun',
 'build',
 'teleporters',
 'find',
 'hair',
 'dresser',
 'spider',
 'cavern',
 'get',
 'sword',
 'shoot',
 'cat',
 'take',
 'lord',
 'moon',
 'use',
 'yoyo',
 'summon',
 'sharknado',
 'minion',
 'shoot',
 'sharks',
 'enemy',
 'find',
 'sky',
 'temple',
 'air',
 'wyverns',
 'spawn',
 'buy',
 'music',
 'box',
 'wizard',
 'go',
 'record',
 'music',
 'like',
 'play',
 'base',
 'whenever',
 'want',
 'go',
 'build',
 'castle',
 'make',
 'entirely',
 'white',
 'marble',
 'would',
 'seem',
 'thing',
 'minecraft',
 'game',
 'dimension',
 'trust',
 'get',
 'use',
 'start',
 'learning',
 'game',
 'terrarium',
 'simply',
 'one',
 'satisfy',
 'sandbox',
 'experience',
 'may',
 'sound',
 'rude',
 'compare',
 'minecraft',
 'imagination']

In [14]:
# filter empty list of strings in X_lemmatized, as they are not useful for topic modelling

X_lemmatized = list(filter(lambda x: len(x) > 0, X_lemmatized))

In [ ]:
# load a custom stopword list

In [ ]:
# add a sklearn CountVectorizer and select top 2000 words for tm
# ref: https://stackoverflow.com/questions/21552518/using-scikit-learn-vectorizers-and-vocabularies-with-gensim

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1, 1), stop_words='english', max_features=2000000)      # default value of max_features in gensim.corpora.Dictionary is 2M
corpus_vect = vect.fit_transform(X_lemmatized)

corpus = gensim.matutils.Sparse2Corpus(corpus_vect, documents_columns=False)

# transform scikit vocabulary into gensim dictionary
id2word = gensim.Dictionary.from_corpus(
    corpus_vect,
    id2word=dict((id, word) for word, id in vect.vocabulary_.items())
)

In [15]:
# use gensim to build a dictionary and train our LDAModel

# it will create a dictionary object, to extract the words from the corpus (i.e. creating tokens)
# id2word = gensim.corpora.Dictionary(X_lemmatized, prune_at=2000000)     # 2M for the default value of the func

id2word.filter_extremes(no_below=50)
# then calling doc2bow on the dictionary object to create a (word (i.e. token), frequency) mapping.
# corpus = [id2word.doc2bow(text) for text in X_lemmatized]

In [45]:
# save the lematized data, as separate pickle file

X_lemmatized_file = dataset_path.parent.joinpath('cleaned_lemmatized', dataset_path.stem + '_cleaned_lemmatized.pkl')

if not X_lemmatized_file.parent.exists():
    X_lemmatized_file.parent.mkdir()

with open(X_lemmatized_file, "wb") as f:
    pickle.dump(X_lemmatized, f)

In [46]:
# load the lematized data, as separate pickle file
# for convenient hyperparameter selection

# X_lemmatized_file = dataset_path.parent.joinpath('cleaned_lemmatized', dataset_path.stem + '_cleaned_lemmatized.pkl')

# with open(X_lemmatized_file, "rb") as f:
#     X_lemmatized = pickle.load(f)

# X_lemmatized[0]

Grid Search

In [16]:
def _print_message(message):
    '''Print message with a timestamp in front of it

    Timestamp format: YYYY-MM-DD HH:MM:SS,mmm
    '''
    print(f'{datetime.now().strftime("%Y-%m-%d %H:%M:%S,%f")[:-3]} - {message}')

In [17]:
def _init_LdaMulticore_params(corpus=None, num_topics=100, id2word=None, workers=None, chunksize=2000, 
        passes=1, batch=False, alpha='symmetric', eta=None, decay=0.5, offset=1.0, 
        eval_every=10, iterations=50, gamma_threshold=0.001, random_state=None, 
        minimum_probability=0.01, minimum_phi_value=0.01, per_word_topics=False, dtype=np.float32):
    
    hyperparameters = dict()
    hyperparameters['corpus'] = corpus
    hyperparameters["num_topics"] = num_topics
    hyperparameters['id2word'] = id2word
    hyperparameters["workers"] = workers
    hyperparameters["chunksize"] = chunksize
    hyperparameters["passes"] = passes
    hyperparameters["alpha"] = alpha
    hyperparameters["eta"] = eta
    hyperparameters["decay"] = decay
    hyperparameters["offset"] = offset
    hyperparameters["eval_every"] = eval_every
    hyperparameters["iterations"] = iterations
    hyperparameters["gamma_threshold"] = gamma_threshold
    hyperparameters['minimum_probability'] = minimum_probability
    hyperparameters["random_state"] = random_state
    hyperparameters['minimum_phi_value'] = minimum_phi_value
    hyperparameters['per_word_topics'] = per_word_topics
    hyperparameters['dtype'] = dtype

    if "alpha" in hyperparameters:
        if isinstance(hyperparameters["alpha"], float):
            hyperparameters["alpha"] = [
                hyperparameters["alpha"]
            ] * hyperparameters["num_topics"]

    return hyperparameters

In [18]:
sys.path.append('../')

from eval_metrics import compute_inverted_rbo, compute_topic_diversity, compute_pairwise_jaccard_similarity, \
                        METRICS, SEARCH_BEHAVIOUR, COHERENCE_MODEL_METRICS

In [19]:
def _init_config_dict(config_path:Path, model_name:str, hyperparameters:dict, search_space_dict:dict, 
                      metrics:list[METRICS], monitor:METRICS,
                      search_behaviour:SEARCH_BEHAVIOUR, search_rs:int, search_n_iter:int):
    # init dict for config.json

    _hyperparameters = hyperparameters
    _search_space_dict = search_space_dict

    if not config_path.exists():
        config = {}
        config['model'] = model_name
        config.update(hyperparameters)

        config.pop('corpus', '')
        config.pop('id2word', '')
        
        # remove hyperparameters that are in the search space
        for key in search_space_dict.keys():
            config.pop(key, '')

        config['dtype'] = str(config['dtype'])      # datatype is not json serializable, so convert to str

        # store the search behaviour
        config['search_behaviour'] = search_behaviour.value

        if search_behaviour == SEARCH_BEHAVIOUR.RANDOM_SEARCH:
            config['search_rs'] = search_rs
            config['search_n_iter'] = search_n_iter

        # store the search space
        config['search_space'] = search_space_dict

        # store the metrics types
        config['metrics'] = list(map(lambda x: x.value, metrics))

        # store the monitor metric
        config['monitor'] = monitor.value

        config['gensim_version'] = str(gensim.__version__)

        # save the file
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)

        _print_message('Created config.json at: ' + str(config_path))
        # print('Created config.json at:', config_path)
    else:
        with open(config_path, 'r') as f:
            config = json.load(f)

        # check whether search behaviour in _search_behaviour is same in _config
        if 'search_behaviour' not in config.keys():
            raise Exception('search_behaviour is not found in config.json. Please modify the hyperparameters passed in.')
        
        if config['search_behaviour'] != search_behaviour.value:
            raise Exception(f'search_behaviour is different in config.json. Please modify the hyperparameters passed in.')
        
        if config['search_behaviour'] == SEARCH_BEHAVIOUR.RANDOM_SEARCH.value:
            if 'search_rs' not in config.keys():
                raise Exception('search_rs is not found in config.json. Please modify the hyperparameters passed in.')
            
            if config['search_rs'] != search_rs:
                raise Exception(f'search_rs is different in config.json. Please modify the hyperparameters passed in.')
            
            if 'search_n_iter' not in config.keys():
                raise Exception('search_n_iter is not found in config.json. Please modify the hyperparameters passed in.')
            
            if config['search_n_iter'] != search_n_iter:
                raise Exception(f'search_n_iter is different in config.json. Please modify the hyperparameters passed in.')

        # check whether hyperparameters in _hyperparameters are same in _config
        # for every key in _hyperparameters, check whether the value is the same in _config
        for key in config.keys() & _hyperparameters.keys():
            if key == 'dtype':      # edge case for dtype
                # convert the _hyperparameters[key] to str for comparison only
                if str(_hyperparameters[key]) != config[key]:
                    raise Exception(f'hyperparameters {key} is different in config.json. Please modify the hyperparameters passed in.')

            else:
                if _hyperparameters[key] != config[key]:
                    raise Exception(f'hyperparameters {key} is different in config.json. Please modify the hyperparameters passed in.')
            
        # check search space
        if 'search_space' not in config.keys():
            raise Exception('search_space is not found in config.json. Please modify the hyperparameters passed in.')
        
        # check whether search space in _search_space_dict is same in _config
        # for every key in _search_space_dict, check whether the value is the same in _config
        for key in config['search_space'].keys() & _search_space_dict.keys():
            if _search_space_dict[key] != config['search_space'][key]:
                raise Exception(f'search_space {key} is different in config.json. Please modify the hyperparameters passed in.')
            
        # check the metrics and monitor
        if 'metrics' not in config.keys():
            raise Exception('metrics is not found in config.json. Please modify the hyperparameters passed in.')
        
        # check whether metrics in _metrics is same in _config
        # for every key in _metrics, check whether the value is the same in _config
        if config['metrics'] != list(map(lambda x: x.value, metrics)):
            raise Exception(f'metrics is different in config.json. Please modify the hyperparameters passed in.')
        
        # check whether monitor in _monitor is same in _config
        # for every key in _monitor, check whether the value is the same in _config
        if config['monitor'] != monitor.value:
            raise Exception(f'monitor is different in config.json. Please modify the hyperparameters passed in.')


        _print_message('Loaded existing config.json from: ' + str(config_path))
        _print_message('Hyperparameters and search space are checked to be consistent with config.json')
        # print('Loaded existing config.json from:', config_path)
        # print('Hyperparameters and search space are checked to be consistent with config.json')

    return config

In [20]:
def _init_result_dict(result_path:Path, monitor_type:str):
    # init dict for result.json

    if not result_path.exists():
        result = {}
        result['best_metric'] = -float('inf')
        result['best_model_checkpoint'] = ""
        result['best_hyperparameters'] = dict()
        result["monitor_type"] = monitor_type
        result["log_history"] = list()
    else:
        with open(result_path, 'r') as f:
            result = json.load(f)

        # check whether metric_type in result.json is same as metric_type passed in
        if 'monitor_type' not in result.keys():
            raise Exception('metric_type is not found in result.json. Please modify the metric_type passed in.')
        elif result['monitor_type'] != monitor_type:
            raise Exception(f'metric_type is different in result.json. Please modify the metric_type passed in.')

        _print_message('Loaded existing result.json from: ' + str(result_path))
        _print_message('metric_type is checked to be consistent with result.json')
        # print('Loaded existing result.json from:', result_path)
        # print('metric_type is checked to be consistent with result.json')

    return result

In [21]:
# referencing octis to calculate topics, topic-document-matrix and topic-word-matrix

def _get_topic_word_matrix(model):
    return model.get_topics()

def _get_topics(model, id2word, result, top_words=10):
    if top_words > 0:
        topics_output = []
        for topic in result["topic-word-matrix"]:
            top_k = np.argsort(topic)[-top_words:]
            top_k_words = list(reversed([id2word[i] for i in top_k]))
            topics_output.append(top_k_words)

        return topics_output
    
def _get_topic_document_matrix(model, corpus, num_topics):
    """
    Return the topic representation of the
    corpus
    """
    doc_topic_tuples = []
    for document in corpus:
        doc_topic_tuples.append(
            model.get_document_topics(document, minimum_probability=0))

    topic_document = np.zeros((
        num_topics,
        len(doc_topic_tuples)))

    for ndoc in range(len(doc_topic_tuples)):
        document = doc_topic_tuples[ndoc]
        for topic_tuple in document:
            topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
    return topic_document

In [22]:
def _load_lda_model(model_path:Path):
    if not model_path.exists():
        raise Exception(f'Cannot find model checkpoint at {model_path}')
    else:
        lda_model = gensim.models.ldamodel.LdaModel.load(str(model_path.joinpath('lda_multicore')))
        return lda_model

In [23]:
# from itertools import product
from gensim.models import CoherenceModel
from copy import deepcopy

from sklearn.model_selection import ParameterGrid, ParameterSampler

def model_search(X, hyperparameters:dict, search_space:dict, save_folder:Path, 
                metrics:list[METRICS]=[METRICS.C_NPMI], monitor:METRICS=METRICS.C_NPMI, 
                save_each_models=True, run_from_checkpoints=False,
                search_behaviour=SEARCH_BEHAVIOUR.GRID_SEARCH, search_rs=42, search_n_iter=10):
    """
    Perform grid search for LDA model hyperparameter selection

    Parameters:
    ----------
    X : List of input texts
    hyperparameters : dict of hyperparameters
    search_space : dict of search space for hyperparameters
    save_each_models : save each model or not
    save_path : folder to save the model
    run_from_checkpoints : whether to run from checkpoints or not

    Returns:
    -------
    best_model : best model
    best_model_path : path to the best model
    best_hyperparameters : best hyperparameters
    """

    config_json_path = save_folder.joinpath('config.json')
    result_json_path = save_folder.joinpath('result.json')

    if monitor not in metrics:
        raise Exception('monitor is not in metrics. Please modify the metrics passed in.')

    if run_from_checkpoints:
        if not save_folder.exists():
            _print_message('Save folder:' + str(save_folder.resolve()) + ' does not exist. Function terminates.')
            # print('Save folder:' + str(save_folder.resolve()) + ' does not exist. Function terminates.')
            raise Exception('No checkpoints found. Function terminates.')
        
        # check for existing configs
        if not config_json_path.exists():
            raise Exception('No config.json found. Function terminates.')
        
        # check for existing results
        if not result_json_path.exists():
            _print_message('No result.json is found. Assuming no existing checkpoints.')
            # print('No result.json is found. Assuming no existing checkpoints.')
    else:
        if save_folder.exists():
            raise Exception('Checkpoints found. Please delete the checkpoints or set run_from_checkpoints=True. Function terminates.')
        
    if not save_folder.exists():
        save_folder.mkdir()

    # init / load existing json files
    # also doing consistency checks for hyperparameters and search space
    config = _init_config_dict(config_json_path, 'lda_multicore', hyperparameters, search_space, metrics, monitor,
                               search_behaviour, search_rs, search_n_iter)

    result = _init_result_dict(result_json_path, monitor.value)
    
    
    _print_message(f'Search folder: {save_folder}')
    # print(f'Search folder: {save_folder}')


    # init
    best_model_path = result['best_model_checkpoint']
    best_metric_score = result['best_metric']
    best_model = _load_lda_model(Path(best_model_path)) if best_model_path != "" else None
    best_hyperparameters = result['best_hyperparameters']


    _print_message(f'Best model checkpoint: {best_model_path}')
    _print_message(f'Best metric score: {best_metric_score}')
    _print_message(f'Best model: {best_model}')
    # print(f'Best model checkpoint: {best_model_path}')
    # print(f'Best metric score: {best_metric_score}')
    # print(f'Best model: {best_model}')

    # use sklearn to generate the search space instead of generating my myself
    if search_behaviour == SEARCH_BEHAVIOUR.GRID_SEARCH:
        search_iterator = ParameterGrid(search_space)
    elif search_behaviour == SEARCH_BEHAVIOUR.RANDOM_SEARCH:
        search_iterator = ParameterSampler(search_space, n_iter=search_n_iter, random_state=search_rs)
    
    print('\n')

    for search_space_dict in search_iterator:

        # create the model folder by using search_space keys and current hyperparams values
        model_name = '_'.join([f'{key}_{value}' for key, value in search_space_dict.items()])

        model_path = save_folder.joinpath(
            'lda_multicore_' + model_name
        )

        # check whether the current search space is already trained
        # by comparing the folder name

        if model_path.exists():
            print(f'Skipping current search space: {search_space_dict}')
            continue


        ##########
        # Training starts
        ##########

        _print_message(f'Training with current search space: {search_space_dict}')
        # print(f'Training with current search space: {search_space_dict}')

        # update existing hyperparams dict
        hyperparameters.update(search_space_dict)

        # train the model
        model = gensim.models.ldamulticore.LdaMulticore(**hyperparameters)

        ##########
        # Training ends
        ##########

        ##########
        # Evaluation starts
        ##########

        result_octis = {}
        result_octis['topic-word-matrix'] = _get_topic_word_matrix(model)
        result_octis['topics'] = _get_topics(model, hyperparameters['id2word'], result_octis, top_words=10)
        result_octis['topic-document-matrix'] = _get_topic_document_matrix(model, corpus, hyperparameters['num_topics'])

        _print_message('Computing evaluation metrics')
        # print('Computing evaluation metrics')

        metrics_score = dict()

        # compute various metrics
        for metric in metrics:
            if metric in COHERENCE_MODEL_METRICS:
                # compute the coherence
                coherencemodel = CoherenceModel(model=model, texts=X, dictionary=id2word, coherence=metric.value)
                score = coherencemodel.get_coherence()                

            elif metric == METRICS.TOPIC_DIVERSITY:
                # compute the coherence
                score = compute_topic_diversity(result_octis, topk=10)

            elif metric == METRICS.INVERTED_RBO:
                # compute the coherence
                score = compute_inverted_rbo(result_octis, topk=10)

            elif metric == METRICS.PAIRWISE_JACCARD_SIMILARITY:
                # compute the coherence
                score = compute_pairwise_jaccard_similarity(result_octis, topk=10)

            else:
                raise Exception(f'Unknown metric: {metric.value}')
            
            metrics_score[metric.value] = score

            _print_message(f'Evaluation metric ({metric.value}): {score}')
            # print(f'Evaluation metric ({metric.value}): {score}')
            
        # get the monitor score
        monitor_score = metrics_score[monitor.value]

        ##########
        # Evaluation ends
        ##########

        ##########
        # Save models
        ##########

        if not model_path.exists():
            model_path.mkdir(parents=True)

        # save the model
        if save_each_models:
            model.save(str(model_path.joinpath('lda_multicore')))

            _print_message('Model saved at: ' + str(model_path))
            print('Model saved at:', model_path)

        ##########
        # Save models ends
        ##########
            
        ###########
        # Update result dict and json file
        ###########
            
        # init

        model_hyperparameters = deepcopy(hyperparameters)
        model_hyperparameters.pop('corpus', '')     # pop as it is not json serializable
        model_hyperparameters.pop('id2word', '')    # pop as it is not json serializable
        model_hyperparameters['dtype'] = str(model_hyperparameters['dtype'])
            
        if monitor_score > best_metric_score:
            best_metric_score = monitor_score
            best_model = model
            best_model_path = model_path
            best_hyperparameters = model_hyperparameters
            
        # update
            
        model_log_history = dict()
        model_log_history.update(metrics_score)         # add the metrics score values to the log history
        model_log_history['model_name'] = model_name
        model_log_history['hyperparameters'] = model_hyperparameters

        result['best_metric'] = best_metric_score
        result['best_model_checkpoint'] = str(best_model_path)      # relative path
        result['best_hyperparameters'] = best_hyperparameters
        result["log_history"].append(model_log_history)

        # print(result)

        # save result
        with open(result_json_path, 'w') as f:
            json.dump(result, f, indent=2)

        _print_message('Saved result.json at: ' + str(result_json_path))
        # print("Saved result.json at:", result_json_path)
        print('\n\n')

    _print_message('Search ends')
    # print('Search ends')
    return best_model, best_model_path, best_hyperparameters


In [25]:
# grid search / random search

hyperparameters = _init_LdaMulticore_params(
    corpus=corpus, num_topics=20, id2word=id2word, 
    workers=3, chunksize=2024, random_state=42, passes=5)

# create search_space dict
search_space = dict()

search_space['num_topics'] = [10, 20]
# search_space['decay'] = [0.7, 0.8, 0.9]
# search_space['offset'] = [16, 64, 128]

search_behaviour = SEARCH_BEHAVIOUR.GRID_SEARCH
# search_behaviour = SEARCH_BEHAVIOUR.RANDOM_SEARCH


training_datetime = datetime.now()
# training_datetime = datetime(2024, 1, 19, 11, 44, 53)

training_folder = Path(f'lda_multicore_{search_behaviour.value}_{training_datetime.strftime("%Y%m%d_%H%M%S")}')

best_model, best_model_path, best_hyperparameters = model_search(
    X_lemmatized, hyperparameters, search_space, 
    training_folder,
    metrics=[METRICS.C_NPMI, METRICS.C_V, METRICS.UMASS, METRICS.C_UCI, METRICS.TOPIC_DIVERSITY, METRICS.INVERTED_RBO, METRICS.PAIRWISE_JACCARD_SIMILARITY],
    monitor=METRICS.C_NPMI,
    search_behaviour=search_behaviour, 
    # search_rs=42, 
    # search_n_iter=10,
    run_from_checkpoints=False)

2024-01-30 16:43:26,475 - Created config.json at: lda_multicore_grid_search_20240130_164326/config.json
2024-01-30 16:43:26,476 - Search folder: lda_multicore_grid_search_20240130_164326
2024-01-30 16:43:26,476 - Best model checkpoint: 
2024-01-30 16:43:26,476 - Best metric score: -inf
2024-01-30 16:43:26,476 - Best model: None


2024-01-30 16:43:26,476 - Training with current search space: {'num_topics': 10}
2024-01-30 16:43:53,037 - Computing evaluation metrics
2024-01-30 16:44:02,009 - Evaluation metric (c_npmi): 0.015634950114236927
2024-01-30 16:44:10,818 - Evaluation metric (c_v): 0.5005047221786995
2024-01-30 16:44:11,816 - Evaluation metric (u_mass): -2.83701467999183
2024-01-30 16:44:20,416 - Evaluation metric (c_uci): -0.31160543475794117
2024-01-30 16:44:20,416 - Evaluation metric (topic_diversity): 0.75
2024-01-30 16:44:20,418 - Evaluation metric (inverted_rbo): 0.8694496359077778
2024-01-30 16:44:20,419 - Evaluation metric (pairwise_jaccard_similarity): 0.06392806635324698

In [ ]:
# N_TOPICS = 20

# # Online LDA, how to effective train LDA models
# # https://papers.nips.cc/paper_files/paper/2010/hash/71f6278d140af599e06ad9bf1ba03cb0-Abstract.html

# lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
#                                              id2word=id2word,
#                                              num_topics=N_TOPICS,         # later can use grid search to find the best number of topics
#                                              random_state=42,
#                                              chunksize=2048,                # chunk size affects memory consumption, and updating speed (like DL batch_size). https://groups.google.com/g/gensim/c/FE7_FYSconA
#                                              passes=10,                     # no. of passes over the whole corpus. If larger chunksize, then the passes should be larger too.
#                                             #  alpha='auto',
#                                              workers=3)     # workers = no. of cores (physical cores, but not logical threads)

---

load the best model from grid search

but also load the id2word object and the corpus corresponding to the model (for separate evaluation and inference)

https://stackoverflow.com/questions/60840809/gensim-how-to-load-corpus-from-saved-lda-model

In [27]:
# load the best model from training folder

training_folder = Path(f'lda_multicore_grid_search_{training_datetime.strftime("%Y%m%d_%H%M%S")}')
training_result_json_path = training_folder.joinpath('result.json')
with open(training_result_json_path, 'r') as f:
    training_result = json.load(f)

best_model_checkpoint_path = Path(training_result['best_model_checkpoint'])

best_id2word = gensim.corpora.Dictionary.load(str(best_model_checkpoint_path.joinpath('lda_multicore.id2word')))
best_corpus = [id2word.doc2bow(text) for text in X_lemmatized]      # recreate the corpus given the id2word (gensim Dictionary)
best_model = gensim.models.ldamulticore.LdaMulticore.load(str(best_model_checkpoint_path.joinpath('lda_multicore')))

print('Best model checkpoint path:', best_model_checkpoint_path)

lda_model = best_model
id2word = best_id2word
corpus = best_corpus


Best model checkpoint path: lda_multicore_grid_search_20240130_164326/lda_multicore_num_topics_10


visualize the data

In [28]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.067534  0.201313       1        1  28.026347
8     -0.145865  0.031390       2        1  13.795712
6     -0.130033  0.245266       3        1  13.514286
1     -0.066946  0.113664       4        1  13.348451
7     -0.264221  0.153866       5        1  13.046183
9      0.252270  0.135880       6        1   8.644870
0      0.161209 -0.074485       7        1   3.902303
3     -0.230721 -0.262877       8        1   2.143332
5      0.335549 -0.151771       9        1   1.822201
4      0.021224 -0.392247      10        1   1.756316, topic_info=           Term           Freq          Total Category  logprob  loglift
15         game  106126.000000  106126.000000  Default  10.0000  10.0000
27    minecraft   20814.000000   20814.000000  Default   9.0000   9.0000
173       world    9568.000000    9568.000000  Default   8.0000   8.0000
32         play   30148.000000   30148.000000  Default   7.0000   7.0000
16          get   23164.000000   23164.000000  Default   6.0000   6.0000
...         ...            ...            ...      ...      ...      ...
1768       step     312.814609     431.726789  Topic10  -4.4607   3.7198
764      pretty     774.273887    3033.985184  Topic10  -3.5544   2.6762
15         game     954.909407  106126.723313  Topic10  -3.3447  -0.6688
243        good     497.693500   13622.773103  Topic10  -3.9964   0.7324
1424       yeah     275.061525     633.178028  Topic10  -4.5893   3.2082

[281 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
12262      3  0.944153         ad
9503       3  0.986363   adictive
554        1  0.989025  aesthetic
5942       7  0.972351      agian
75         1  0.178350      alone
...      ...       ...        ...
405        6  0.950147     zombie
405        7  0.002320     zombie
405        9  0.010441     zombie
2411       1  0.003636       zone
2411       8  0.992565       zone

[1127 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 7, 2, 8, 10, 1, 4, 6, 5])

save model

we need to save the corpora.Dictionary and the LDA model

In [ ]:
# save the LDA multicore model (and the corpora.Dictionary object) automatically

# lda_save_folder = Path(f'lda_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
# if not lda_save_folder.exists():
#     lda_save_folder.mkdir()

# lda_model.save(str(lda_save_folder.joinpath('lda_model')))     # no need to add file extension

Evaluation

gensim provide functions to calculate, so we don't need to install octis (as the evaluation backend of octis also relies on gensim)

octis seems awesome for simple development, but it installs many packages ;(

In [ ]:
# corpus = lemmatized words (?) (list of list of str)

# create a result object from the LDAMulticore model for octis evaluation
# referencing from https://github.com/MIND-Lab/OCTIS/blob/master/octis/models/LDA.py
# and guideline in README: https://github.com/MIND-Lab/OCTIS/tree/master
result_lda_online = {}
result_lda_online['topic-word-matrix'] = lda_model.get_topics()

top_words = 10
topics_output = []
for topic in result_lda_online["topic-word-matrix"]:
    top_k = np.argsort(topic)[-top_words:]
    top_k_words = list(reversed([id2word[i] for i in top_k]))
    topics_output.append(top_k_words)
result_lda_online["topics"] = topics_output

def _get_topic_document_matrix(lda_model, corpus, num_topics=10):
    """
    Return the topic representation of the
    corpus
    """

    id_corpus = corpus

    doc_topic_tuples = []
    for document in id_corpus:
        doc_topic_tuples.append(
            lda_model.get_document_topics(document, minimum_probability=0))

    topic_document = np.zeros((num_topics, len(doc_topic_tuples)))

    for ndoc in range(len(doc_topic_tuples)):
        document = doc_topic_tuples[ndoc]
        for topic_tuple in document:
            topic_document[topic_tuple[0]][ndoc] = topic_tuple[1]
    return topic_document

result_lda_online['topic-document-matrix'] = _get_topic_document_matrix(lda_model, corpus, num_topics=N_TOPICS)

In [ ]:
lda_model.show_topics(num_topics=N_TOPICS, num_words=10, formatted=True, log=False)

In [ ]:
# setup: get the model's topics in their native ordering...
all_topics = lda_model.print_topics()
# ...then create a empty list per topic to collect the docs:
docs_per_topic = [[] for _ in all_topics]

# now, for every doc...
for doc_id, doc_bow in enumerate(corpus):
    # ...get its topics...
    doc_topics = lda_model.get_document_topics(doc_bow)
    # ...& for each of its topics...
    for topic_id, score in doc_topics:
        # ...add the doc_id & its score to the topic's doc list
        docs_per_topic[topic_id].append((doc_id, score))

In [ ]:
# If you're interested in the top docs per topic, you can further sort each list's pairs by their score

for doc_list in docs_per_topic:
    doc_list.sort(key=lambda id_and_score: id_and_score[1], reverse=True)

In [ ]:
print(docs_per_topic[0][:10])

In [ ]:
# show top 10 documents for each topic, also the name of the game
for topic_id, docs in enumerate(docs_per_topic):
    print(f'Topic {topic_id + 1}:')
    for doc_id, score in docs[:10]:
        print(f'Game: {dataset.iloc[doc_id]["app_name"]}')
        print(f'Doc ID: {doc_id}')
        print(f'Score: {score}')
        print(f'Doc: {dataset.iloc[doc_id]["review_text"]}')
        print()
    print('\n\n\n\n\n')

In [ ]:
dataset.iloc[1655473]

In [ ]:
X[1655473]

In [ ]:
result_lda_online['topic-document-matrix'][0]

In [ ]:
lda_model.get_topics().shape

In [ ]:
np.sum(result_lda_online['topic-document-matrix'], axis=0)

INFERENCE

inference test

In [ ]:
# inference test

inference_test = ["well its been fun guys, but that's it, no more updates, that one was the last one, there is no longer going to be anymore content for this game anymore, there is no way to replay it as there won't be any updates, nope, that was it, the last update, nothing more, this game has no new ways to experience it as there is no more content updates, nothing new to freshen up the experience, its such a shame that this game has no replay-ability, once you beat the game there is like no point to playing again, as they said guys 1.2 will be they final update. nothing more after 1.2, there is no chance they will make another final update right? several years and final updates later: alright, thats it, no more updates we wont be getting anymore, thats it, nothing more, no more updates, for real this time... oh god, redigit made another tweet.",
                  "keeps forcing me to play it",
'''I will leave the cat here, so that everybody who passes by can pet it and give it a thumbs up and awards
　　　 　　／＞　　フ
　　　 　　| 　_　 _ l
　 　　 　／` ミ＿xノ
　　 　 /　　　 　 |
　　　 /　 ヽ　　 ﾉ
　 　 │　　|　|　|
　／￣|　　 |　|　|
　| (￣ヽ＿_ヽ_)__)
　＼二つ''']

inference_test = cleaning_strlist(inference_test)

inference_test = list(map(lambda x: lemmatization(x), inference_test))

corpus_test = [id2word.doc2bow(text) for text in inference_test]

test_output = lda_model[corpus_test]

test_output

In [ ]:
inference_test[-1]

In [ ]:
# test inference

corpus_test = [id2word.doc2bow(text) for text in inference_test]

output_test = lda_model[corpus_test]

for i in range(len(output_test)):
    # print(sorted(test_output[i], key=lambda x: x[1], reverse=True))
    print(sorted(output_test[i], key=lambda x: x[1], reverse=True))

load model (both corpora Dictionary and the LDA model)

In [ ]:
# del id2word
# del lda_model

# model_datetime = datetime(2024, 1, 15, 0, 21, 57)
# lda_save_folder = Path(f'lda_model_{model_datetime.strftime("%Y%m%d_%H%M%S")}')

# # id2word_load = gensim.corpora.Dictionary.load('lda_model.id2word')
# id2word_l = gensim.corpora.Dictionary.load(str(lda_save_folder.joinpath('lda_model.id2word')))

# lda_model_l = gensim.models.ldamulticore.LdaMulticore.load(str(lda_save_folder.joinpath('lda_model')))

In [ ]:
# corpus_test2 = [id2word_l.doc2bow(text) for text in inference_test]

# output_test2 = lda_model_l[corpus_test2]

# for i in range(len(output_test2)):
#     print(sorted(output_test2[i], key=lambda x: x[1], reverse=True))